In [9]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import numpy as np

import time
import os
import random
import string
from cms.CountMinSketch import CountMinSketch


def Update_SS(Sk_head,element):
    if Sk_head.get(element.ID):
        Sk_head[element.ID]+=element.count
    else:
        if len(Sk_head)<TC.width:
            Sk_head[element.ID]=element.count
        else:
            # Sk_head is full
            '''
            ## space saving
            # Sk_head is full, replace min
            min_ID = min(Sk_head, key=Sk_head.get)
            Sk_head[element.ID]=Sk_head[min_ID]+element.count
            Sk_head.pop(min_ID)
            '''
            temp_dict=Sk_head.copy()
            for i in temp_dict:
                Sk_head[i]-=1
                if Sk_head[i]<=0:
                    Sk_head.pop(i)


filename='kosarak.dat'
filepath=r"..\..\dataset\kosarak"
gr_file_name='kosarak_ground_truth.csv'
src_data=os.path.join(filepath,filename)
    # dataset
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth            
                        
w=2688
d=8
size=1024
topk=1024
TC.Set_default(w,d,size,topk)
    # set width, depth, size of T, random seed of hash
    # Config.width, Config.depth

cms=CountMinSketch(TC.width,TC.depth)
Sk_head=dict()
Sketch=np.zeros((TC.depth,TC.width),dtype='int32')
Top_dict=dict()
e_min=DS.Tail("",1)

#stream_size=50
#input_stream=[random.choice(string.ascii_lowercase) for _ in range(stream_size)]

#ground_truth=dict()
#item_count=0

start=time.time()
with open(src_data,'r') as file:
    while True:
        #item_count+=1
        #print("\n{}-th, {} arrives:".format(item_count,e))
        e=file.readline().strip('\n')
        if not e:
            print('EOF')
            break
        else:
            '''
            # update ground truth        
            if ground_truth.get(item.ID):
                ground_truth[item.ID]+=1
            else:
                ground_truth[item.ID]=1        
            '''
            item=DS.Tail(e,1)
            # Stream processing
            if Top_dict.get(item.ID):
                # e in Top
                #print("{} hits in Top:".format(e))
                Top_dict[item.ID]+=1
            else:
                # e is not in Top
                if len(Top_dict)<TC.size:
                    #print("Top is not full")
                    Top_dict[item.ID]=1
                else:
                    #print("Top is full and not hit")
                    Update_SS(Sk_head,item)
                    cms.Add_CMS(item)
                    if Sk_head[item.ID]>TC.e_max.count:
                        TC.e_max.ID=item.ID
                        TC.e_max.count=Sk_head[item.ID]
                    if TC.e_max.count>e_min.count:
                        min_ID = min(Top_dict, key=Top_dict.get)
                        e_min=DS.Tail(min_ID,Top_dict[min_ID])
                        #print("count of {} > {}".format(item.ID,e_min.ID))
                        e_est=min(cms.Estimate_CMS(item),TC.e_max.count)
                        # estmiated freq of e_max in Sketch

                        #print("{}_CMS_estimate = {}".format(item.ID,cms.Estimate_CMS(item)))
                        #print("{}_SS_estimate = {}".format(item.ID,Sk_head[item.ID]))
                        if e_est>e_min.count:
                            #print("Bring back")
                            e_pop=DS.Tail(TC.e_max.ID,-e_est)

                            # Bring e_max to Top and pop e_min
                            Top_dict[TC.e_max.ID]=e_est
                            Top_dict.pop(min_ID)

                            # add e_min to Sketch and reduce e_max 
                            cms.Add_CMS(e_min)
                            cms.Add_CMS(e_pop)
                            Sk_head.pop(e_pop.ID)
                            TC.e_max.ID=""
                            TC.e_max.count=0
                            Update_SS(Sk_head,e_min)                
            #print("Ground_truth: {}".format(ground_truth))
            #print("Top: {}".format(Top_dict))
            #print("Sk_head:{}".format(Sk_head))
            #print("CMS:\n{}".format(cms.sketch))
end=time.time()
print("Execution time:{:8.3f} seconds.".format(end-start))

KeyError: '6956 4868'

In [ ]:
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
tp_set,precision=TF.Get_precision(gr_path,Top_dict,TC.topk)
    # send default top k
print("Precision: {:6.3f}".format(precision))

In [10]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import numpy as np

import time
import os
import random
import string


stream_size=100
input_stream=[random.choice(string.ascii_lowercase) for _ in range(stream_size)]

ground_truth=dict()
item_count=0


w=8
d=4
size=5
topk=5
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Sk_head=[DS.Head(0) for j in range(TC.depth)]
Sketch=np.zeros((TC.depth,TC.width),dtype='int32')
Top_dict=dict()
e_min=DS.Tail("",1)

start=time.time()
for e in input_stream:
    #item_count-=1
    item=DS.Tail(e,1)
    if Top_dict.get(item.ID):
        # e in Top
        Top_dict[item.ID]+=1
    else:
        if len(Top_dict)<TC.size:
            Top_dict[item.ID]=1
        else:
            TF.UpdateSk(item,Sk_head,Sketch)
            print(Sk_head)
            print(e_max)
    if TC.e_max.count>e_min.count:
        min_ele = min(Top_dict, key=Top_dict.get)
            # 找dict中的最小key元素
        e_min=DS.Tail(min_ele,Top_dict[min_ele])
        TF.BringBack(e_min,Top_dict,Sk_head,Sketch)
end=time.time()

'''
snapshot = TF.tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')
    # stop track memory usage===========================================================
'''

print("Top-{},Sketch:{}*{}".format(TC.size,TC.depth,TC.width))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Element-Precision
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
tp_set,precision=TF.Get_precision(gr_path,Top_dict,TC.topk)
    # send default top k
print("Precision: {:6.3f}".format(precision))


'''
# display memory usage
TF.display_top(snapshot)
'''
print("Top_dict with {} bytes.".format(sys.getsizeof(Top_dict)))
print("Sketch with {} bytes.".format(sys.getsizeof(Sketch)))
sk1=sys.getsizeof(Sk_head[0].distinct.__slots__)
sk2=sys.getsizeof(Sk_head[0].count)
sk3=sys.getsizeof(Sk_head[0].maxID)
sk4=sys.getsizeof(Sk_head[0].keep)
sk_sum=sum([sk1,sk2,sk3,sk4])*TC.depth
print("Sk_head with {} bytes.".format(sk_sum))
print("Total size {} bytes, ".format(sys.getsizeof(Top_dict)+sys.getsizeof(Sketch)+sk_sum))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,Top_dict,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 
TF.Plot_topk_compare(gr_path,Top_dict,"Cyclic Sketch")

# All element compare
cardinality=[len(i.distinct) for i in Sk_head]
    # list for all row cardinality
ground_truth=TF.Get_ground_truth(gr_path)
My_result=dict()
for item in ground_truth:
    My_result[item]=TF.Query9(item,Top_dict,Sk_head,Sketch,cardinality)

TF.Plot_all_compare(ground_truth,My_result,"Cyclic Sketch")

IndexError: index 5 is out of bounds for axis 0 with size 4